## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-13-19-08-52 +0000,nypost,San Francisco schools chief ripped for ‘crocod...,https://nypost.com/2026/02/13/us-news/san-fran...
1,2026-02-13-19-08-18 +0000,bbc,Two Britons among three dead in French Alps av...,https://www.bbc.com/news/articles/cd03elmng7do...
2,2026-02-13-19-08-07 +0000,bbc,Six possible effects of Trump's climate policy...,https://www.bbc.com/news/articles/cd03ee39945o...
3,2026-02-13-19-07-23 +0000,wapo,January inflation eased more than expected in ...,https://www.washingtonpost.com/business/2026/0...
4,2026-02-13-19-05-10 +0000,nyt,How Bangladesh Went From Revolution to Elections,https://www.nytimes.com/video/world/1000000107...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2534/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,34
109,epstein,21
276,new,14
118,world,12
86,ice,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
337,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,93
175,2026-02-13-12-00-00 +0000,wsj,Republican Sen. Rand Paul joined Democrats in ...,https://www.wsj.com/politics/gop-sen-rand-paul...,85
218,2026-02-13-10-00-16 +0000,wapo,Services cut. Staff laid off. New Trump cuts h...,https://www.washingtonpost.com/education/2026/...,76
280,2026-02-13-01-44-10 +0000,latimes,Judge blocks Trump administration move to cut ...,https://www.latimes.com/politics/story/2026-02...,70
50,2026-02-13-17-52-55 +0000,nypost,Trump says he plans to visit Venezuela after C...,https://nypost.com/2026/02/13/us-news/trump-sa...,67


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
337,93,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...
307,66,2026-02-12-23-16-31 +0000,nypost,Here’s the real reason Goldman CEO David Solom...,https://nypost.com/2026/02/12/business/heres-t...
56,66,2026-02-13-17-39-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
89,55,2026-02-13-16-18-00 +0000,wsj,What a shutdown would mean for the Department ...,https://www.wsj.com/politics/policy/homeland-s...
66,46,2026-02-13-17-11-14 +0000,latimes,San Francisco teachers' strike ends as union a...,https://www.latimes.com/california/story/2026-...
301,42,2026-02-12-23-33-00 +0000,wsj,When Customs and Border Protection fired a las...,https://www.wsj.com/politics/national-security...
100,39,2026-02-13-15-57-09 +0000,nypost,Watch out for these 5 hidden hotel charges — a...,https://nypost.com/2026/02/13/lifestyle/watch-...
27,37,2026-02-13-18-35-50 +0000,nyt,"Amid Fallout From Epstein Files, Dubai’s DP Wo...",https://www.nytimes.com/2026/02/13/world/middl...
158,36,2026-02-13-13-04-33 +0000,nypost,Trans shooter Jesse Van Rootselaar — who kille...,https://nypost.com/2026/02/13/world-news/jesse...
280,33,2026-02-13-01-44-10 +0000,latimes,Judge blocks Trump administration move to cut ...,https://www.latimes.com/politics/story/2026-02...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
